# Train Dashboard

Duckdb Dbhelper.py


In [ ]:
import duckdb
import streamlit as st

class DB:
    def __init__(self, db_path="train_tickets.duckdb"):
        try:
            self.conn = duckdb.connect(db_path)
            self.cursor = self.conn.cursor()
            print("✅ DuckDB connected")
        except Exception as e:
            print("❌ Connection failed:", e)

    def fetch_station_names(self):
        station = []
        self.cursor.execute("""
            SELECT DISTINCT(source) FROM train_tickets
            UNION
            SELECT DISTINCT(destination) FROM train_tickets
        """)
        data = self.cursor.fetchall()
        for row in data:
            station.append(row[0])
        return station

    def search_tickets(self, source, destination):
        query = f"""
            SELECT train_id, train_name, class, days_of_operation FROM train_tickets
            WHERE source = '{source}' AND destination = '{destination}'
        """
        self.cursor.execute(query)
        data = self.cursor.fetchall()
        column_names = [desc[0] for desc in self.cursor.description]
        result = [dict(zip(column_names, row)) for row in data]
        return result

    def class_frequency(self):
        labels = []
        values = []
        self.cursor.execute("""
            SELECT class, COUNT(*) FROM train_tickets
            GROUP BY class
        """)
        data = self.cursor.fetchall()
        for row in data:
            labels.append(row[0])
            values.append(row[1])
        return labels, values

    def station_wise_booking(self):
        stations = []
        bookings = []
        self.cursor.execute("""
            SELECT source, COUNT(*) AS Bookings
            FROM train_tickets
            GROUP BY source
            ORDER BY Bookings DESC
        """)
        data = self.cursor.fetchall()
        for row in data:
            stations.append(row[0])
            bookings.append(row[1])
        return stations, bookings

    def daily_bookings(self):
        dates = []
        counts = []
        self.cursor.execute("""
            SELECT travel_date, COUNT(*) AS Bookings FROM train_tickets
            GROUP BY travel_date
            ORDER BY travel_date
        """)
        data = self.cursor.fetchall()
        for row in data:
            dates.append(str(row[0]))
            counts.append(row[1])
        return dates, counts

    def revenue_over_time(self):
        rev_dates = []
        rev_values = []
        self.cursor.execute("""
            SELECT travel_date, SUM(price) as total_revenue FROM train_tickets
            GROUP BY travel_date
            ORDER BY travel_date
        """)
        data = self.cursor.fetchall()
        for row in data:
            rev_dates.append(str(row[0]))
            rev_values.append(row[1])
        return rev_dates, rev_values


# app.py

In [ ]:
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
from dbhelper_duckdb import DB  # Use DuckDB version
from datetime import datetime

# Initialize DB
db = DB()

# Sidebar
st.sidebar.image("train_quotes.jpg", use_container_width=True)
st.sidebar.title("🚆 Train Ticket Dashboard")
menu = st.sidebar.selectbox("Menu", options=['Select One', 'Check Tickets', 'Analytics'])

# Check Tickets Section
if menu == 'Check Tickets':
    st.title('Search Train Tickets')

    stations = db.fetch_station_names()

    col1, col2 = st.columns(2)
    with col1:
        source = st.selectbox('Source Station', sorted(stations))
    with col2:
        destination = st.selectbox('Destination Station', sorted(stations))

    if st.button('Search'):
        results = db.search_tickets(source, destination)
        if results:
            st.subheader(f"Tickets from {source} to {destination}")
            st.table(results)
        else:
            st.warning('No tickets found.')

# Analytics Section
elif menu == 'Analytics':
    st.title("📊 Booking Analytics")

    st.subheader('Filters')
    all_classes = ['Sleeper', 'AC 2 Tier', 'AC 3 Tier', 'AC First Class']
    selected_class = st.multiselect('Select Class', options=all_classes, default=None)
    start_date = st.date_input("Start Date", datetime(2023, 1, 1))
    end_date = st.date_input("End Date", datetime(2023, 12, 31))

    # Pie Chart - Class Frequency
    labels, values = db.class_frequency()
    if labels and values:
        fig1, ax1 = plt.subplots()
        ax1.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
        ax1.axis('equal')
        st.subheader('Class Distribution')
        st.pyplot(fig1)
        st.markdown(f"✅ **Conclusion:** The most booked class is **{labels[values.index(max(values))]}**, indicating a strong preference among passengers.")

    # Bar Chart - Station Wise Bookings
    stations, bookings = db.station_wise_booking()
    if stations and bookings:
        fig2, ax2 = plt.subplots()
        sns.barplot(x=bookings, y=stations, ax=ax2, palette='viridis')
        ax2.set_xlabel("Number of Bookings")
        ax2.set_ylabel("Source Station")
        st.subheader('Top Boarding Stations')
        st.pyplot(fig2)
        st.markdown(f"📍 **Conclusion:** The station with the highest number of departures is **{stations[0]}**, highlighting its role as a major boarding hub.")

    # Line Chart - Daily Bookings
    dates, counts = db.daily_bookings()
    if dates and counts:
        fig3, ax3 = plt.subplots()
        sns.lineplot(x=dates, y=counts, ax=ax3, color='green')
        ax3.set_xlabel("Date")
        ax3.set_ylabel("No. of Bookings")
        plt.xticks(rotation=75)
        st.subheader('📅 Bookings Over Time')
        st.pyplot(fig3)
        st.markdown("📈 **Conclusion:** The booking trend shows peaks and troughs, indicating seasonal or event-based travel patterns.")

    # Line Chart - Revenue Over Time
    st.subheader("💰 Revenue Over Time")
    rev_dates, rev_values = db.revenue_over_time()
    if rev_dates and rev_values:
        fig4, ax4 = plt.subplots()
        sns.lineplot(x=rev_dates, y=rev_values, ax=ax4, color='orange')
        ax4.set_xlabel("Date")
        ax4.set_ylabel("Total Revenue")
        plt.xticks(rotation=85)
        st.pyplot(fig4)
        st.markdown("💹 **Conclusion:** Revenue follows a trend similar to bookings, with noticeable spikes likely driven by peak travel days.")

        # Peak Booking Day
        st.subheader("🔥 Peak Booking Day")
        max_index = rev_values.index(max(rev_values))
        peak_day = rev_dates[max_index]
        st.success(f"Highest Revenue Day: {peak_day} with ₹{rev_values[max_index]:,}")

# Welcome Section
else:
    st.title("Welcome to the Train Ticket Dashboard 🚉")
    st.image("train_logo.jpg", use_container_width=True)
    st.markdown("""
        ## Features
        - 🔍 Search tickets by source & destination
        - 📊 Visual analytics (class distribution, top stations, booking trends)
        - 💰 Revenue insights and peak booking dates
        - 📅 Filters by class and date range

        ### Built with ❤️ by **Kaustav Roy Chowdhury** using Streamlit, Seaborn, DuckDB
    """)


# ✅ Full Updated Section of app.py (top portion only)

In [ ]:
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import duckdb
from dbhelper_duckdb import DB  # Use DuckDB version
from datetime import datetime

# 🔹 Load CSV and register as in-memory DuckDB table
df = pd.read_csv("train_tickets.csv")
conn = duckdb.connect("train_tickets.duckdb")
conn.register("train_tickets", df)


# Optional: persist table if not done before
# conn.execute("CREATE TABLE IF NOT EXISTS train_tickets AS SELECT * FROM train_tickets")

conn.close()

# 🔹 Initialize DB class
db = DB()


In [ ]:
conn.execute("SELECT COUNT(*) FROM train_tickets").fetchall()


In [ ]:
conn.execute("CREATE TABLE IF NOT EXISTS train_tickets AS SELECT * FROM train_tickets")


# Videogame Dashboard

In [ ]:
import duckdb

class DB:
    def __init__(self, db_path="video_game_sales.duckdb"):
        try:
            self.conn = duckdb.connect(db_path)
            self.cursor = self.conn.cursor()
            print("✅ DuckDB connected")
        except Exception as e:
            print("❌ Connection failed:", e)

    def fetch_genres(self):
        self.cursor.execute("SELECT DISTINCT genre FROM video_game_sales ORDER BY genre")
        return [row[0] for row in self.cursor.fetchall()]

    def fetch_publishers(self):
        self.cursor.execute("SELECT DISTINCT publisher FROM video_game_sales ORDER BY publisher")
        return [row[0] for row in self.cursor.fetchall()]

    def fetch_platforms(self):
        self.cursor.execute("SELECT DISTINCT platform FROM video_game_sales ORDER BY platform")
        return [row[0] for row in self.cursor.fetchall()]

    def fetch_years(self):
        self.cursor.execute("SELECT DISTINCT year FROM video_game_sales WHERE year IS NOT NULL ORDER BY year")
        return [row[0] for row in self.cursor.fetchall()]

    def genre_sales_by_year(self, year):
        self.cursor.execute("""
            SELECT genre, SUM(global_sales)
            FROM video_game_sales
            WHERE year = ?
            GROUP BY genre
            ORDER BY SUM(global_sales) DESC
        """, (year,))
        data = self.cursor.fetchall()
        return [row[0] for row in data], [row[1] for row in data]

    def publisher_sales_by_year(self, year):
        self.cursor.execute("""
            SELECT publisher, SUM(global_sales)
            FROM video_game_sales
            WHERE year = ?
            GROUP BY publisher
            ORDER BY SUM(global_sales) DESC
            LIMIT 20
        """, (year,))
        data = self.cursor.fetchall()
        return [row[0] for row in data], [row[1] for row in data]

    def platform_sales_by_year(self, year):
        self.cursor.execute("""
            SELECT platform, SUM(global_sales)
            FROM video_game_sales
            WHERE year = ?
            GROUP BY platform
            ORDER BY SUM(global_sales) DESC
        """, (year,))
        data = self.cursor.fetchall()
        return [row[0] for row in data], [row[1] for row in data]

    def sales_over_time(self):
        self.cursor.execute("""
            SELECT year, SUM(global_sales)
            FROM video_game_sales
            WHERE year IS NOT NULL
            GROUP BY year
            ORDER BY year
        """)
        data = self.cursor.fetchall()
        return [row[0] for row in data], [row[1] for row in data]
